In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
#data.isnull().sum()




#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)


# Specify the columns you want to standardize
columns_to_standardize = ["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]

# Create a StandardScaler object
scaler = StandardScaler()

# Loop through the columns and standardize each one
for column in columns_to_standardize:
    data[column] = scaler.fit_transform(data[[column]])



import torch
import torch.nn as nn
import pandas as pd

# Sample data with a 'date' column containing dates from 1980-12-12 to 2023-07-31
data2 = pd.DataFrame({'date': pd.date_range(start='1980-12-12', end='2023-07-31')})

# Convert 'date' column to datetime
data2['date'] = pd.to_datetime(data['date'])

# Extract day, month, and year from the date column
data['day'] = data2['date'].dt.day
data['month'] = data2['date'].dt.month
data['year'] = data2['date'].dt.year

# Define the embedding dimensions
embedding_dim = 1  # You can adjust this dimension as needed

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor(data['day'].values)
month_tensor = torch.LongTensor(data['month'].values)
year_tensor = torch.LongTensor(data['year'].values - 1980)  # Convert years to an index from 0 to 43

# Pass tensors through embedding layers to get embeddings
day_embeddings = day_embedding(day_tensor)
month_embeddings = month_embedding(month_tensor)
year_embeddings = year_embedding(year_tensor)

# Concatenate the embeddings
date_embeddings = torch.cat((day_embeddings, month_embeddings, year_embeddings), dim=1)

# Print the resulting embeddings
print(date_embeddings)




tensor([[ 1.6939,  0.5628, -0.8295],
        [-1.4341,  0.5628, -0.8295],
        [ 0.9223,  0.5628, -0.8295],
        ...,
        [ 0.2628,  1.0180, -0.3356],
        [-0.8428,  1.0180, -0.3356],
        [ 1.6939,  1.0180, -0.3356]], grad_fn=<CatBackward0>)


In [7]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)

# Convert the combined testing data to PyTorch tensors
x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)






# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        # Define the LSTM layer as a class attribute
        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out




# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 16
n_layers = 4
sequence_length = 75  # Keep this as 1 for your input data
batch_size = 64

# Concatenate date embeddings with your feature vectors
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)




# Convert the combined data to PyTorch tensors
#x_train_tensor = torch.tensor(x_train_combined, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train, dtype=torch.float32)



# Copy x_train_combined to create x_train_tensor
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)


# Create y_train_tensor directly from y_train
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)





# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)




# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
learning_rate = 0.00015
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Sequence Length={sequence_length}, Batch Size={batch_size}, Input Size={input_size}, Date Embedding Dim={date_embedding_dim}, Hidden Dim={hidden_dim},'
              f'Layers={n_layers}')


# Training loop
num_epochs = 6000

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Extract features on the testing set
        model.eval()
        with torch.no_grad():
            # Extract features from the hidden states
            hidden_states, _ = model.lstm(x_test_tensor.view(x_test_tensor.size(0), 1, -1))
            val_outputs = model(x_test_tensor)
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
        # You can now use 'hidden_states' as the feature representations of your sequences
        # The shape of 'hidden_states' will be (batch_size, sequence_length, hidden_dim)

# Print hidden_states after training
print(hidden_states)










C:\Users\pitr7\AppData\Local\Temp\ipykernel_10824\3780818392.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)


Hyperparameters: Learning Rate=0.00015, Sequence Length=75, Batch Size=128, Input Size=7, Date Embedding Dim=3, Hidden Dim=32,Layers=4
Epoch [100/6000], Loss: 20179.1074, Val Loss: 24988.3164
Epoch [200/6000], Loss: 17991.1426, Val Loss: 21851.9277
Epoch [300/6000], Loss: 15801.0273, Val Loss: 19602.1719
Epoch [400/6000], Loss: 20484.6602, Val Loss: 16422.4590
Epoch [500/6000], Loss: 21904.3828, Val Loss: 14199.4062
Epoch [600/6000], Loss: 10129.4570, Val Loss: 12343.0645
Epoch [700/6000], Loss: 8227.5293, Val Loss: 10745.6104
Epoch [800/6000], Loss: 13269.9443, Val Loss: 9357.6357
Epoch [900/6000], Loss: 11669.9980, Val Loss: 8135.4287
Epoch [1000/6000], Loss: 12349.7695, Val Loss: 7039.7686
Epoch [1100/6000], Loss: 11019.9355, Val Loss: 6053.5967
Epoch [1200/6000], Loss: 7551.7412, Val Loss: 5169.3735
Epoch [1300/6000], Loss: 3663.1045, Val Loss: 4381.0874
Epoch [1400/6000], Loss: 1889.1721, Val Loss: 3679.4456
Epoch [1500/6000], Loss: 4219.6245, Val Loss: 3060.9858
Epoch [1600/6000]

In [12]:
# 1. Feature Extraction

model.eval()
with torch.no_grad():
    hidden_states, _ = model.lstm(x_test_tensor.view(x_test_tensor.size(0), 1, -1))
    val_outputs = model(x_test_tensor)
    y_test_predictions = val_outputs


# 2. generate date embeddings for the future date, 11th August

import torch

# Define the target date: 12th August
target_date = "11th August"

# Define the embedding dimensions (same as in your previous code)
embedding_dim = 1

# Define the maximum values for day, month, and year based on your previous code
max_day = 31  # Maximum day
max_month = 12  # Maximum month
max_year = 43  # Maximum year (from 1980 to 2023)

# Create one-hot encodings for day, month, and year
day_encoding = torch.zeros(max_day)
month_encoding = torch.zeros(max_month)
year_encoding = torch.zeros(max_year + 1)  # +1 to account for the inclusive range

# Map the target date to one-hot encoding
# Extract the day and month from the target date
day_index = int(target_date.split(" ")[0].replace("th", "")) - 1  # Extract the day from the target date
month_index = 8  # August is the 9th month (0-based index)

# Set the corresponding elements to 1
day_encoding[day_index] = 1
month_encoding[month_index] = 1
year_encoding[43] = 1  # 43 corresponds to the year 2023 in your previous code

# Concatenate the day, month, and year encodings to get the date embedding
date_embedding_11_august = torch.cat((day_encoding, month_encoding, year_encoding), dim=0)

# 3. Combine LSTM Features with Date Embeddings



import torch

# Assuming 'hidden_states' has shape [3547, 1, 32]
# 'date_embedding_11_august' has shape [76]

# Broadcast 'date_embedding_11_august' to match the shape of 'hidden_states'
# This will repeat 'date_embedding_11_august' along the second dimension
date_embedding_11_august_broadcasted = date_embedding_11_august.reshape(1, 1, -1).expand(3547, 1, -1)

# Combine 'hidden_states' and 'date_embedding_11_august_broadcasted'
combined_states = torch.cat((hidden_states, date_embedding_11_august_broadcasted), dim=2)

# The resulting 'combined_states' will have a shape of [3547, 1, 32 + 76]



RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 3557 but got size 3547 for tensor number 1 in the list.